In [1]:
import json
from http import HTTPStatus
import dashscope
from dotenv import dotenv_values
from retry import retry
config = dotenv_values('.env')

In [2]:
MODEL_NAME = 'qwen-max'
dashscope.api_key = config['qwen_key'],

@retry(delay=60, tries=3)
def call_qwen_api(MODEL_NAME, query):
    messages = [
        {'role': 'user', 'content': query}]
    response = dashscope.Generation.call(
        MODEL_NAME,
        messages=messages,
        result_format='message',  # set the result is message format.
    )
    if response.status_code == HTTPStatus.OK:
        #print(response)
        return response['output']['choices'][0]['message']['content']
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
        raise Exception()

In [3]:
def get_prompt(problem, question, options):

    options = '\n'.join(f"{'ABCDEFG'[i]}. {o}" for i, o in enumerate(options))

    prompt = f"""你是一个逻辑推理专家，擅长解决逻辑推理问题。以下是一个逻辑推理的题目，形式为单项选择题。所有的问题都是（close-world assumption）闭世界假设，即未观测事实都为假。请逐步分析问题并在最后一行输出答案，最后一行的格式为"答案是：A"。题目如下：

### 题目:
{problem}

### 问题:
{question}
{options}
"""

    return prompt

In [4]:
def get_question_prompt(data):
    prompt = f"""你是一个逻辑推理问题出题专家，以下json是一个problem，对应一些子问题。除了这些子问题外再生成三个其他子问题，保持格式一致并给出参考答案。
### promble如下：
{data}
### 输出格式如下:
{{
promble:
questions:[
question:
options:[]
]
}}
"""
    return prompt


In [5]:
def read_file(ifn):
    # 读取输入文件
    data = []
    with open(ifn) as reader:
        for line in reader:
            # 解析每一行JSON数据
            sample = json.loads(line)
            data.append(sample)

    return data

In [6]:
def write_jsonl(data):
    with open('data/output.jsonl', 'w', encoding='utf-8') as file:
        for item in data:
            json.dump(item, file)
            file.write('\n')

In [7]:
import re


def extract_json(response):
    # 使用正则表达式匹配 JSON 数据
    match = re.search(r'```json\s*(.*?)\s*```', response, re.DOTALL)
    if match:
        # 如果找到了 JSON 数据，则返回清理后的 JSON 字符串
        return match.group(1).strip()
    else:
        # 如果没有找到 JSON 数据，则返回原始响应
        return response


def process_data(data):
    query = get_question_prompt(data)
    respond = ''
    try:
        respond = call_qwen_api(MODEL_NAME, query)
        # 清理 JSON 字符串
        extract_json(respond)
        # 解析 JSON 字符串
        data = json.loads(respond)
    except Exception as e:
        print(respond)
        print(f"Error: {str(e)}")
    return data

In [8]:
file = read_file('data/round1_train_data.jsonl')

In [9]:
from tqdm import tqdm
import concurrent.futures
file = file[:5]

# 并行批量处理
def batch_process_questions(file, max_workers=5):
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_data, data) for data in file]
        results = [future.result() for future in tqdm(concurrent.futures.as_completed(futures), total=len(file))]
    return results

results = batch_process_questions(file)

write_jsonl(results)

 20%|██        | 1/5 [00:32<02:10, 32.73s/it]

```json
{
"problem": "有一个方程的系数分别为A, B和C，下面是一些已知的规则：\n\n1. 如果B^2 - 4AC = 0，则方程有一个实数解。\n2. 如果B^2 - 4AC > 0，则方程有两个不同的实数解。\n3. 如果B^2 - 4AC < 0，则方程有两个不同的复数解。\n\n根据以上规则，回答以下选择题：",
"questions": [
{"question": "选择题 1：\n当系数A=7, B=9, C=5时，方程的解是什么？", "options": ["-1.2857142857142858", "0", "+(-0.6428571428571429, *(0.5486532677049005, i)) 和 -(-0.6428571428571429, *(0.5486532677049005, i))", "2.2857142857142856"], "answer": "C"},
{"question": "选择题 2：\n当系数A=7, B=9, C=5时，方程是否有解为6？", "options": ["是", "否"], "answer": "B"},
{"question": "选择题 3：\n当系数A=7, B=9, C=5时，方程是否有解为8？", "options": ["是", "否"], "answer": "B"},
{"question": "选择题 4：\n当系数A=7, B=6, C=0时，方程是否有解为6？", "options": ["是", "否"], "answer": "B"},
{"question": "选择题 5：\n如果一个方程的解为两个相同的实数解，那么B^2 - 4AC应该等于什么？", "options": ["0", "正数", "负数"], "answer": "0"},
{"question": "选择题 6：\n当系数A=1, B=2, C=1时，方程的解是实数还是复数？", "options": ["实数", "复数"], "answer": "实数"},
{"question": "选择题 7：\n如果B^2 - 4AC = 25，这说明方程有几个实数解？", "options": ["0", "1", "2"], "answer": "2"}
],
"

 40%|████      | 2/5 [00:34<00:43, 14.51s/it]

```json
{
"problem": "有一组程序可以用来计算两个数的最小公倍数（LCM）。该程序使用了最大公约数（GCD）来帮助计算。具体过程如下：\n\n1. 如果两个数相等，则它们的GCD就是其中任意一个数。\n2. 如果第一个数小于第二个数，则用第二个数减去第一个数，并对新结果递归地计算GCD。\n3. 如果第一个数大于第二个数，则交换两个数并递归地计算GCD。\n\n基于此GCD的计算，我们可以通过以下公式计算两个数的LCM：\n\\[ \\text{LCM}(x, y) = \\frac{x}{\\text{GCD}(x, y)} \\times y \\]\n\n根据上述原理，请回答以下选择题：",
"questions": [
{"question": "选择题 1：\n已知一个数和9的最小公倍数是9，这个数是多少？", "options": ["1", "3", "6", "9"], "answer": "D"},
{"question": "选择题 2：\n7和2的最小公倍数是8，这个说法正确吗？", "options": ["正确", "错误", "取决于7和2的最大公约数", "以上都不对"], "answer": "B"},
{"question": "选择题 3：\n3和9的最小公倍数是3，这个说法正确吗？", "options": ["正确", "错误", "取决于3和9的最大公约数", "以上都不对"], "answer": "B"},
{"question": "选择题 4：\n3和4的最小公倍数是3，这个说法正确吗？", "options": ["正确", "错误", "取决于3和4的最大公约数", "以上都不对"], "answer": "B"},
{"question": "选择题 5：\n如果两个数的最大公约数是1，它们的最小公倍数等于什么？", "options": ["两数之和", "两数之积", "两数中较大的那个", "无法确定"], "answer": "B"},
{"question": "选择题 6：\n计算15和25的最大公约数后，利用该GCD计算它们的最小公倍数应该是多少？", "options": ["375", "75", "150", "50"], "answer": "A"},
{"questi

 60%|██████    | 3/5 [00:40<00:21, 10.76s/it]

```json
{
"problem": "在一场山地自行车比赛中，四位选手分别取得了第一、第二、第三和第四名。不同的颜色代表不同的排名。下面是一些关于这场比赛和选手排名的信息：\n\n1. Alan 名列第一。\n2. 第二名的选手穿红色服装。\n3. John 没有穿黄色服装。\n4. 第四名的选手穿蓝色服装。\n5. Steve 的排名和他的服装色是相同的名次。\n6. Kev 的名次排在 Steve 前面。\n7. 第二名的选手穿的不是青色。\n8. 黄色服穿的选手的成绩排在绿色服穿的选手前面。\n9. 确保每四个参数中的所有元素都不相同和符合排名顺序。\n\n根据上述信息， 回答以下选择题：",
"questions": [
{"question": "选择题 1:\n根据比赛结果，排在第二名之后的可能是哪些名次？", "options": ["第一名 第三名", "第三名 第四名"], "answer": "B"},
{"question": "选择题 2:\n第一名是不是在第三名之前？", "options": ["是", "否"], "answer": "A"},
{"question": "选择题 3:\n第一名是不是在第一名之前？", "options": ["是", "否"], "answer": "B"},
{"question": "选择题 4:\n第一名是不是在第四名之前？", "options": ["是", "否"], "answer": "A"},
{"question": "选择题 5:\nSteve 有没有可能穿绿色服装？", "options": ["是", "否"], "answer": "否"},
{"question": "选择题 6:\nKev 的服装颜色能否确定？如果能，是什么颜色？", "options": ["能，红色", "能，黄色", "不能确定"], "answer": "不能确定"},
{"question": "选择题 7:\nJohn 的排名是否一定在第三名或第四名？", "options": ["是", "否"], "answer": "是"}
],
"id": "round1_train_data_002"
}
```

### 参考答案说明：

- **选择题 5**: Steve 的排名和他的

 80%|████████  | 4/5 [00:44<00:07,  7.94s/it]

```json
{
  "problem": "有一个英文到法文的词汇表，包含以下对应词汇：\n\n1. the -> le\n2. cat -> chat\n3. jumps -> sauts\n4. over -> sur\n5. moon -> lune\n6. cow -> vache\n7. plays -> joue\n8. fiddle -> violon\n9. egg -> œuf\n10. falls -> tombe\n11. off -> de\n12. wall -> mur\n\n根据这个词汇表，翻译以下英文句子成法文：",
  "questions": [
    {
      "question": "选择题 1：\n英文句子 \"the cat jumps over the moon\" 翻译成法文是：",
      "options": ["le chat saute sur la lune", "le chat sauts sur le lune", "le sauts chat sur le lune", "le chat sauts sur le lune"],
      "answer": "A"
    },
    {
      "question": "选择题 2：\n英文句子 \"the cow plays the fiddle\" 翻译成法文是：",
      "options": ["le vache jouer le violon", "le jouer vache le violon", "le vache jouer la vièle", "la vache joue le violon"],
      "answer": "D"
    },
    {
      "question": "选择题 3：\n英文句子 \"the egg falls off the wall\" 翻译成法文是：",
      "options": ["le bougre des chutes de le mur", "le oeuf tombe de le mur", "le bougre tomber de le mur", "des chutes le bougre de le mur"],
     

100%|██████████| 5/5 [00:45<00:00,  9.08s/it]

```json
{
"problem": "在安大略省，我们有关于不同城市及它们之间位置关系的信息。我们知道各个城市的人口以及哪些城市在给定的城市西边。以下是我们已知的信息：\n\n- 布兰普顿的人口是590000。\n- 汉密尔顿的人口是540000。\n- 金斯顿的人口是130000。\n- 基奇纳的人口是230000。\n- 伦敦的人口是380000。\n- 马克翰的人口是330000。\n- 密西沙加的人口是720000。\n- 渥太华的人口是930000。\n- 多伦多的人口是2700000。\n- 旺市的人口是310000。\n- 温莎的人口是220000。\n\n并且我们知道以下城市在彼此的西边：\n\n- 温莎在伦敦的西边。\n- 伦敦在汉密尔顿的西边。\n- 汉密尔顿在密西沙加的西边。\n- 密西沙加在多伦多的西边。\n- 多伦多在金斯顿的西边。\n- 金斯顿在渥太华的西边。\n- 旺市在马克翰的西边。\n\n还有些城市位于其他城市的北边：\n\n- 布兰普顿在密西沙加的北边。\n- 基奇纳在伦敦的北边。\n- 旺市在多伦多的北边。\n\n根据以上信息，回答以下选择题：",
"questions": [
{
"question": "选择题 1：\n位于布兰普顿以西的城市是哪一个？",
"options": ["多伦多", "金斯顿", "渥太华", "汉密尔顿"],
"answer": "D"
},
{
"question": "选择题 2：\n哪个城市的人口最多，且直接相邻于人口第二多的城市？",
"options": ["多伦多", "温莎", "密西沙加", "汉密尔顿"],
"answer": "A"
},
{
"question": "选择题 3：\n哪两个城市之间不存在直接的西-东或北-南的位置关系，但通过其他城市间接相连？",
"options": ["基奇纳与温莎", "布兰普顿与金斯顿", "伦敦与渥太华", "旺市与汉密尔顿"],
"answer": "C"
},
{
"question": "选择题 4：\n哪个城市既不在任何其他城市西边也不在北边，同时人口少于500,000？",
"options": ["基奇纳", "金斯顿", "马克翰", "温莎"],
"answer": "B"
}
],
"id": "round1

In [ ]:
extract_json(response)